In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/openit-donnees/donnees.csv")
print("Number of rows: " + str(df.shape[0]))
print("Number of columns: " + str(df.shape[1]))
df.columns

Nombre de ligne et de colonnes, valeurs manquantes par colonnes, type des colonnes, afficher les caractéristiques statistiques avec la fonction .describe()

In [ ]:
df

In [ ]:
#afficher types des colonnes
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
# Remplacer les valeurs manquantes par la moyenne
df.fillna(df.mean(numeric_only=True), inplace=True)

In [ ]:
# Assurez-vous que la colonne "Date_Heure" est au format datetime
df['Date_Heure'] = pd.to_datetime(df['Date_Heure'])

# Créez des colonnes pour l'heure de la journée, le jour, le mois et l'année
df['Hour'] = df['Date_Heure'].dt.hour
df['Day'] = df['Date_Heure'].dt.day
df['Month'] = df['Date_Heure'].dt.month
df['Year'] = df['Date_Heure'].dt.year
df['DayOfWeek'] = df['Date_Heure'].dt.dayofweek


df = df.drop(['index','Periode de mesure de la rafale'], axis=1)

df

In [ ]:
# Define a function to map the month to a season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Hiver'
    elif month in [3, 4, 5]:
        return 'Printemps'
    elif month in [6, 7, 8]:
        return 'Ete'
    else:
        return 'Automne'

# Add a season column to the dataframe
df['Saison'] = df['Month'].map(get_season)


In [ ]:
# Perform one-hot encoding on the Season column
season_dummies = pd.get_dummies(df['Saison'], prefix='Saison')

# Add the one-hot encoded columns to the original dataframe
df = pd.concat([df, season_dummies], axis=1)

df

In [ ]:
#condition = lambda row: 1 if (row['Month'] >= 3) and (row['Month'] < 6) else 0
#df['Printemps'] = df.apply(condition, axis=1)

In [ ]:
#condition = lambda row: 1 if (row['Month'] >= 6) and (row['Month'] < 9) else 0
#df['Ete'] = df.apply(condition, axis=1)

In [ ]:
#condition = lambda row: 1 if (row['Month'] >= 9) and (row['Month'] < 12) else 0
#df['Automne'] = df.apply(condition, axis=1)

In [ ]:
#condition = lambda row: 1 if (row['Month'] == 12) or (row['Month'] < 3) else 0
#df['Hiver'] = df.apply(condition, axis=1)

In [ ]:
condition = lambda row: 1 if (row['DayOfWeek'] >= 0) and (row['DayOfWeek'] <= 4) else 0
df['Normal_Day'] = df.apply(condition, axis=1)

In [ ]:
condition = lambda row: 1 if (row['DayOfWeek'] == 5) or (row['DayOfWeek'] == 6) else 0
df['Weekend'] = df.apply(condition, axis=1)

In [ ]:
condition = lambda row: 1 if (row['Hour'] >= 5) and (row['Hour'] <= 12) else 0
df['Matin'] = df.apply(condition, axis=1)

In [ ]:
condition = lambda row: 1 if (row['Hour'] >= 13) and (row['Hour'] <= 19) else 0
df['Apresmidi'] = df.apply(condition, axis=1)

In [ ]:
condition = lambda row: 1 if (row['Hour'] >= 20) or (row['Hour'] <= 4) else 0
df['Nuit'] = df.apply(condition, axis=1)

df

In [ ]:
#df.corr()

In [ ]:
#Afficher la heatmap des corrélations avec le paramètre : annot=true
#sns.set(rc={'figure.figsize':(30,8)})
#sns.heatmap(df.corr(), annot=True,cmap='RdBu')
#plt.savefig("heatmap.png") 
#plt.show()

In [ ]:
#df = pd.get_dummies(df, columns=['Printemps', 'Ete', 'Automne', 'Hiver', 'Matin', 'Apresmidi', 'Nuit'])
#df

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
#df.loc[:, ['Pression au niveau mer',
#       'Variation de pression en 3 heures', 'consommation']]

In [ ]:
sns.boxplot(x="Saison_Automne", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Saison_Ete", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Saison_Hiver", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Saison_Printemps", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Normal_Day", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Weekend", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="DayOfWeek", y="consommation", data=df, palette='rainbow')

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x='Rafales sur une période',y='consommation',data=df)
plt.xlabel('Rafales sur une période')
plt.ylabel('consommation')

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x='Vitesse du vent moyen 10 mn',y='consommation',data=df)
plt.xlabel('Vitesse du vent moyen 10 mn')
plt.ylabel('consommation')

In [ ]:
#columns = ['Pression au niveau mer', 'Vitesse du vent moyen 10 mn', 'Point de rosée', 'Température (°C)', 'Hour', 'Day', 'Month', 'consommation']

#sns.pairplot(df[columns])
#plt.savefig("pair_plot.png")

In [ ]:
sns.boxplot(x="Day", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Month", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Hour", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Matin", y="consommation", data=df, palette='rainbow')

In [ ]:
sns.boxplot(x="Apresmidi", y="consommation", data=df, palette='rainbow')

In [ ]:
#sns.boxplot(x="Nuit", y="consommation", data=df, palette='rainbow')

In [ ]:
df.dtypes

In [ ]:
sum_by_group = df.groupby(['Day' ,'Month', 'Year'])['consommation'].transform('sum')

df['sum_consommation_by_date'] = sum_by_group

df.tail(7560)

In [ ]:
feature_columns = ['Pression au niveau mer', 'Humidité', 'Température (°C)', 
                   'Saison_Ete', 'Saison_Hiver', 'Saison_Printemps', 'Normal_Day', 
                   'Weekend', 'Day', 'Month', "sum_consommation_by_date"]

df = df[feature_columns]

feature_columns = ['Pression au niveau mer', 'Humidité', 'Température (°C)', 
                   'Saison_Ete', 'Saison_Hiver', 'Saison_Printemps', 'Normal_Day', 
                   'Weekend', 'Hour', 'Month', 'Year', 'Matin', 'Apresmidi', 'consommation']

df = df[feature_columns]

df.columns

In [ ]:
# Split the data
X = df.drop("sum_consommation_by_date", axis=1)
y = df["sum_consommation_by_date"]

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

RandomForestRegressor

In [ ]:
# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(n_estimators=100)

# fit your model
rf_model.fit(X_train, y_train)
rf_model_preds = rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score
mse = mean_squared_error(y_test, rf_model_preds)
# Calculate the mean absolute error of your Random Forest model on the validation data
print('Mean Absolute Error:', mean_absolute_error(y_test, rf_model_preds))
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, rf_model_preds)))

In [ ]:
from sklearn.metrics import r2_score

score=r2_score(y_test,rf_model_preds)
score

In [ ]:

plt.figure(figsize=(10, 5))
plt.scatter(y_test, rf_model_preds, s=20)
plt.title('Predicted vs. Actual')
plt.xlabel('Actual consommation')
plt.ylabel('Predicted consommation')

plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)])
plt.tight_layout()

In [ ]:
# Example weather data for a future day
# Replace these values with the actual consummation data
future_consummation_data = {
    'Pression au niveau mer': [1012],  # Replace with actual data
    'Humidité': [50],
    'Température (°C)': [22],  # Replace with actual data
    'Saison_Ete': [False],
    'Saison_Hiver': [False],
    'Saison_Printemps': [True],
    'Normal_Day': [True],
    'Weekend': [False],
    'Day': [1],
    'Month': [1]
}

# Convert the dictionary to a DataFrame
future_consummation_df = pd.DataFrame.from_dict(future_consummation_data)

# Make predictions using the trained model
future_consummation_preds = rf_model.predict(future_consummation_df)

print(future_consummation_preds)
# Print the predictions
for hour, consummation in enumerate(future_consummation_preds):
        print(f"Hour {hour}: {consummation:.2f} kWh")

LinearRegression

In [ ]:
from sklearn import linear_model

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)  #phase d’apprentissage
lr_pred = lr.predict(X_test)
lr_score=r2_score(y_test,lr_pred)
lr_score               

In [ ]:
# Calculate the mean absolute error of your Random Forest model on the validation data
print('Mean Absolute Error:', mean_absolute_error(y_test, lr_pred))
print('Mean Squared Error:', mean_squared_error(y_test, lr_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, lr_pred)))

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(y_test, lr_pred, s=20)
plt.title('Predicted vs. Actual')
plt.xlabel('Actual consommation')
plt.ylabel('Predicted consommation')

plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)])
plt.tight_layout()